In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
import time
from pandas import DataFrame, read_csv, concat
from tqdm import tqdm
import math

from selenium.webdriver.support.select import Select

# 交易结果公告

In [ ]:
driver = webdriver.Chrome('../chromedriver')
driver.get("http://pwq.hxee.com.cn:7100/news/newest")

total_pages = driver.find_element_by_id("_currentTotalPageNo").text.split('/')[1]
url_list = []
for i in range(int(total_pages)):
    print("page:" + str(i+1))
    urls_tag = driver.find_element_by_class_name("text_l_main").find_elements_by_tag_name("a")
    for tag in urls_tag:
        url_list.append(tag.get_attribute("href"))
#     next page
    driver.find_element_by_id("_toNextPage").click()   
    time.sleep(1)

result = []
header = ["公告编号","交易时间","标的","交易方式","出让方","标的物","标的物数量/吨","标的物期限","总成交量","出让方本次委托已成交数量/吨","受让方公司名称","受让量/吨","受让期限"]
count = 0
for url in url_list:
    driver.get(url)
    print(count)
    time.sleep(1)
    while True:
        try:
            text = driver.find_element_by_class_name("news_details_main").text
        except Exception:
            print('Page reload ... ')
            driver.get(url)
            time.sleep(1)
        else:
            if text != "":
                break
            else:
                print('Page reload ... ')
                driver.get(url)
                time.sleep(1)

    col_1 = text.split("\n")[1].split("编号：")[1].strip().strip("(").strip("）")
    col_2 = text.split('于')[1].split('对')[0]
    col_3 = text.split("《")[1].split("》")[0]
    col_4 = text.split("以")[1].split("的方式")[0]
    col_5 = text.split("出让方：")[1].split("\n")[0]
    if len(text.split("标的物：")[1].split('；')[0].split('，')) == 2:
        col_6 = text.split("标的物：")[1].split('；')[0].split('，')[0].split("共")[0]
        col_7 = text.split("标的物：")[1].split('；')[0].split('，')[0].split("共")[1].split('吨')[0]
        col_8 = text.split("标的物：")[1].split('；')[0].split('，')[1].split('期限')[1]
    else:
        col_6 = text.split("标的物：")[1].split('；')[0].split('，')[0]
        col_7 = text.split("标的物：")[1].split('；')[0].split('，')[1].split('共')[1].split('吨')[0]
        col_8 = text.split("标的物：")[1].split('；')[0].split('，')[2].split('期限')[1]

    if len(text.split("总成交量：")) == 1:
        col_9 = '-'

    else:
        col_9 = text.split("总成交量：")[1].split("；")[0].split('吨')[0].strip()


    if "本日成交" in text:
        col_10 = text.split("本日成交")[1].split('吨')[0].strip()
    elif "全部成交" in text:
        col_10 = "全部成交"
    else:
        col_10 = text.split("已成交")[1].split('吨')[0].strip()

    common_col = [col_1, col_2, col_3,col_4,col_5,col_6,col_7,col_8,col_9, col_10]
    for item in text.split("受让方：")[1].split("；")[:-1]:
        info_1 = item.split('，')[0].split('.')[1]
        info_2 = item.split('，')[1].split("为")[1].split('吨')[0]
        info_3 = item.split('，')[2].split('期限')[1]
        info_row = [info_1, info_2, info_3]
        row = []
        row.extend(common_col)
        row.extend(info_row)

        if len(row) == 13:
            result.append(row)
        else:
            print(len(row))
            print(url)
    count = count + 1       

file_name = "福建省排污权交易中心_交易结果公告_" + time.strftime("%Y-%m-%d", time.localtime()) + ".csv"
DataFrame(result,columns=header).to_csv(file_name)    

# 出让公告

In [ ]:
driver = webdriver.Chrome('../chromedriver')
driver.get("http://pwq.hxee.com.cn:7100/news/tradeout")

time.sleep(1)
total_pages = driver.find_element_by_id("_currentTotalPageNo").text.split('/')[1]
url_list = []
for i in range(int(total_pages)):
    print("page:" + str(i+1))
    urls_tag = driver.find_element_by_class_name("text_l_main").find_elements_by_tag_name("a")
    for tag in urls_tag:
        url_list.append(tag.get_attribute("href"))
#     next page
    driver.find_element_by_id("_toNextPage").click()   
    time.sleep(1)

df_result = DataFrame()
for url in tqdm(url_list):
    driver.get(url)
    time.sleep(1)

    header = ['公告名称','项目编号','标的名称','标的数量','标的来源','出让标的所在地','初始排污权有偿使用费（元/年吨）','挂牌价格（元/年吨）','交易方式','交易开始时间','公告内容']
    result = []
    while True:
        try:
            notice_title = driver.find_element_by_id('notice_title').text
        except Exception:
            print('Page reload ... ')
            driver.get(url)
            time.sleep(1)
        else:
            if notice_title != "":
                break
            else:
                print('Page reload ... ')
                driver.get(url)
                time.sleep(1)    
    
    result = {}
    table_rows = driver.find_element_by_id("Content").find_elements_by_tag_name('tr')
    table_data = []
    result['公告名称'] = notice_title
    for row in table_rows:
        num_data = len(row.find_elements_by_tag_name('td'))
        row_data_el = row.find_elements_by_tag_name('td')
        row_data = []
        for data in row_data_el:
            row_data.append(data.text)

        if num_data == 2:
            result[row_data[0]] = [row_data[1]]
        if num_data == 4:
            result[row_data[0]] = [row_data[1]]
            result[row_data[2]] = [row_data[3]]
        if num_data == 1:
            result['公告内容'] = [row_data[0]]

    df_temp = DataFrame(result)
    df_result = concat([df_result, df_temp], axis=0, ignore_index=True)
file_name = "福建省排污权交易中心_出让公告_" + time.strftime("%Y-%m-%d", time.localtime()) + ".csv"
df_result.to_csv(file_name)

# 受让公告

In [132]:
driver = webdriver.Chrome('../chromedriver')
driver.get("http://pwq.hxee.com.cn:7100/news/tradein")

In [6]:
time.sleep(1)
total_pages = driver.find_element_by_id("_currentTotalPageNo").text.split('/')[1]
url_list = []
for i in tqdm(range(int(total_pages))):
    time.sleep(1)
#     print("page:" + str(i+1))
    urls_tag = driver.find_element_by_class_name("text_l_main").find_elements_by_tag_name("a")
    for tag in urls_tag:
        url_list.append(tag.get_attribute("href"))
#     next page
    driver.find_element_by_id("_toNextPage").click()   
    time.sleep(1)

100%|█████████████████████████████████████████| 944/944 [32:40<00:00,  2.08s/it]


In [133]:
df_result = DataFrame()

for url in tqdm(url_list):
    driver.get(url)
    time.sleep(1)

    result = {}
    while True:
        try:
            notice_title = driver.find_element_by_id('notice_title').text
        except Exception:
            print('Page reload ... ')
            driver.get(url)
            time.sleep(1)
        else:
            if notice_title != "":
                break
            else:
                print('Page reload ... ')
                driver.get(url)
                time.sleep(1)     
    

    createTime = driver.find_element_by_id('createTime_id').text.split('：')[1]

    result['公告名称'] = [notice_title]
    result['发布时间'] = [createTime]

    text = driver.find_element_by_id('Content').text
#     text_span = driver.find_element_by_id('Content').find_elements_by_tag_name('span')
#     if len(text_span) == 0:
#         text_span = driver.find_element_by_id('Content').find_elements_by_tag_name('u')


    result['申购企业'] = [text.split('，')[1].split('拟申购')[0]]

#     num_pollutant = len(text.split('拟申购')[1].split('，期限')[0].split("：")) - 1
    
    
    result['意向时间'] = [text.split('于')[1].split('向')[0]] 
    
    result['期限/年'] = text.split('期限')[1].split('年')[0]
    string_te = text.split('拟申购')[1].split('期限')[0].replace(" ","").replace('：','')

    index = 0
    index_end = 0
    end = len(string_te)
    substance = ''

    while True:
        quant = ''
        if index == end:
            break

        if string_te[index].isdigit() or string_te[index] == '.':
            if not string_te[index+1].isdigit() and string_te[index+1] != '.':
                pass
            else:
                substance = string_te[index_end:index]
    #             print('----')
    #             print(substance)

                while True:
                    if string_te[index] == '吨':
                        break
    #                 print(string_te[index])
                    quant += string_te[index]
                    index += 1  
    #             print(quant)
                index_end = index + 2
#                             print(substance)
            result[substance] = [quant]
#                             print(quant)

        index += 1 
#     for pair in text.split('拟申购')[1].split('，期限')[0].split('，'):
#         if len(pair) != 0:
#             try:
#                 result[pair.split('：')[0]+'/吨'] = [pair.split('：')[1].split('吨')[0]] 
#             except Exception:
# #                 print(text.split('拟申购')[1].split('，期限')[0].split('，'))
# #                 print(url)
#                 string_te = text.split('拟申购')[1].split('期限')[0].replace(" ","").replace('：','')

#                 index = 0
#                 index_end = 0
#                 end = len(string_te)
#                 substance = ''

#                 while True:
#                     quant = ''
#                     if index == end:
#                         break

#                     if string_te[index].isdigit() or string_te[index] == '.':
#                         if not string_te[index+1].isdigit() and string_te[index+1] != '.':
#                             pass
#                         else:
#                             substance = string_te[index_end:index]
#                 #             print('----')
#                 #             print(substance)

#                             while True:
#                                 if string_te[index] == '吨':
#                                     break
#                 #                 print(string_te[index])
#                                 quant += string_te[index]
#                                 index += 1  
#                 #             print(quant)
#                             index_end = index + 2
# #                             print(substance)
#                         result[substance] = [quant]
# #                             print(quant)

#                     index += 1 

#                 continue

        
#     if len(text_span) == 0:
#         result['期限/年'] = text.split('期限')[1].split('年')[0]
#         for pair in text.split('拟申购')[1].split('，期限')[0].split('，'):
#             result[pair.split('：')[0]+'/吨'] = [pair.split('：')[1]]
#     else:
#         result['期限/年'] = [text_span[num_pollutant*2].text]
#         for i in range(0, num_pollutant*2, 2):
#             result[text_span[i].text+'/吨'] = [text_span[i+1].text]




 
    
    df_temp = DataFrame(result)
    df_result = concat([df_result, df_temp], axis=0, ignore_index=True)

  3%|▉                                     | 148/5664 [04:23<2:41:33,  1.76s/it]

Page reload ... 


 30%|██████████▉                          | 1675/5664 [45:02<1:58:21,  1.78s/it]

Page reload ... 


 39%|██████████████▎                      | 2195/5664 [58:44<1:17:52,  1.35s/it]

...

 84%|███████████████████████████████▏     | 4770/5664 [2:06:28<37:27,  2.51s/it]

Page reload ... 


 85%|███████████████████████████████▎     | 4788/5664 [2:07:06<32:12,  2.21s/it]

Page reload ... 


100%|█████████████████████████████████████| 5664/5664 [2:24:34<00:00,  1.53s/it]


In [135]:
file_name = "福建省排污权交易中心_受让公告_" + time.strftime("%Y-%m-%d", time.localtime()) + ".csv"
df_result.to_csv(file_name)